## Clustering K-Mean

In [ ]:
#r "nuget: Microsoft.ML, 5.0.0"
#r "nuget: Microsoft.ML.FastTree, 5.0.0"
#r "nuget: Microsoft.ML.LightGbm, 5.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

Installed Packages Microsoft.ML, 5.0.0 Microsoft.ML.FastTree, 5.0.0 Microsoft.ML.LightGbm, 5.0.0

In [2]:
public class CustomerData
{
    public float MonthlySpend { get; set; }
    public float VisitFrequency { get; set; }
}


In [4]:
public class ClusterPrediction
{
    [ColumnName("PredictedLabel")]
    public uint PredictedClusterId { get; set; }
}


In [3]:
var customers = new List<CustomerData>
{
    new CustomerData { MonthlySpend = 200, VisitFrequency = 2 },
    new CustomerData { MonthlySpend = 220, VisitFrequency = 1 },
    new CustomerData { MonthlySpend = 800, VisitFrequency = 6 },
    new CustomerData { MonthlySpend = 850, VisitFrequency = 7 },
    new CustomerData { MonthlySpend = 400, VisitFrequency = 3 },
    new CustomerData { MonthlySpend = 420, VisitFrequency = 4 }
};


In [6]:
MLContext mlContext = new MLContext();
IDataView dataView = mlContext.Data.LoadFromEnumerable(customers);

In [7]:
// Pipeline for the Model
var pipeline = mlContext.Transforms.Concatenate("Features", nameof(CustomerData.MonthlySpend), nameof(CustomerData.VisitFrequency))
    .Append(mlContext.Clustering.Trainers.KMeans(numberOfClusters: 3));

In [8]:
var model = pipeline.Fit(dataView);

In [9]:
// ClusterPrediction
var predictor = mlContext.Model.CreatePredictionEngine<CustomerData, ClusterPrediction>(model);

In [10]:
var newCustomer = new CustomerData { MonthlySpend = 830, VisitFrequency = 5 };
var prediction = predictor.Predict(newCustomer);
Console.WriteLine($"Predicted Cluster ID: {prediction.PredictedClusterId}");

Predicted Cluster ID: 2


In [11]:
foreach (var customer in customers)
{
    var pred = predictor.Predict(customer);
    Console.WriteLine($"Customer with Monthly Spend: {customer.MonthlySpend}, Visit Frequency: {customer.VisitFrequency} is in Cluster: {pred.PredictedClusterId}");
}

Customer with Monthly Spend: 200, Visit Frequency: 2 is in Cluster: 3
Customer with Monthly Spend: 220, Visit Frequency: 1 is in Cluster: 3
Customer with Monthly Spend: 800, Visit Frequency: 6 is in Cluster: 2
Customer with Monthly Spend: 850, Visit Frequency: 7 is in Cluster: 2
Customer with Monthly Spend: 400, Visit Frequency: 3 is in Cluster: 1
Customer with Monthly Spend: 420, Visit Frequency: 4 is in Cluster: 1
